### Why memory is important?
    - chat models totally rely on memory
    - memory is the key to understand the context
    
### How to handle memory on LLMs?
    1. put it in the prompt
    2. have external lookup

### What is the best way to handle memory?
    - put it in the prompt is easier but is not the best way, because as conversation goes on, the prompt will be too long
      but we can recosntruct the memory from the conversation in different ways to optimize the memory as,
            1. summarize the conversation
            2. buffer the conversation
            3. buffer the conversation with window  
            4. buffer the conversation with window and summarization
            5. use knowledge graph 

In [42]:
import yaml, os, openai
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.prompts.prompt import PromptTemplate
from langchain.chains.conversation.memory import ConversationBufferMemory, \
                                                 ConversationSummaryMemory, \
                                                 ConversationBufferWindowMemory, \
                                                 ConversationSummaryBufferMemory, \
                                                 ConversationKGMemory, ConversationEntityMemory

In [6]:
with open('cadentials.yaml') as f:
    credentials = yaml.load(f, Loader=yaml.FullLoader)

os.environ['OPENAI_API_KEY'] = credentials['OPENAI_API_KEY']
os.environ['HUGGINGFACEHUB_API_TOKEN'] = credentials['HUGGINGFACEHUB_API_TOKEN']
os.environ['ENGINE'] = credentials['ENGINE']

openai.api_key = credentials['OPENAI_API_KEY']
openai.api_base = credentials['OPENAI_API_BASE']
openai.api_type = credentials['OPENAI_API_TYPE']
openai.api_version = credentials['OPENAI_API_VERSION']
openai.engine = credentials['ENGINE']

In [3]:
llm = ChatOpenAI(
                openai_api_key=os.environ["OPENAI_API_KEY"],
                engine = os.environ["ENGINE"],
                model='gpt-3.5-turbo',
                temperature=0.9, 
                max_tokens = 256
                )

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


## ConversationBufferMemory

In [7]:
memory = ConversationBufferMemory()

conversation = ConversationChain(
                                llm=llm, 
                                verbose=True, 
                                memory=memory
                                )

In [8]:
conversation.predict(input="Hi there! I am Zuu")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there! I am Zuu
AI:

> Finished chain.


"Hello, Zuu! I'm an AI designed to have conversations with people. How can I assist you today?"

In [9]:
conversation.predict(input="Do you know machine learning?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there! I am Zuu
AI: Hello, Zuu! I'm an AI designed to have conversations with people. How can I assist you today?
Human: Do you know machine learning?
AI:

> Finished chain.


'Yes, I am familiar with machine learning. Machine learning is a subset of artificial intelligence that focuses on the development of algorithms and models that allow computers to learn and make predictions or decisions without being explicitly programmed. It involves the use of large amounts of data to train models and improve their performance over time. Machine learning is used in a wide range of applications, such as image and speech recognition, natural language processing, recommendation systems, and autonomous vehicles, to name just a few.'

In [10]:
conversation.predict(input="Name few algorithms ?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there! I am Zuu
AI: Hello, Zuu! I'm an AI designed to have conversations with people. How can I assist you today?
Human: Do you know machine learning?
AI: Yes, I am familiar with machine learning. Machine learning is a subset of artificial intelligence that focuses on the development of algorithms and models that allow computers to learn and make predictions or decisions without being explicitly programmed. It involves the use of large amounts of data to train models and improve their performance over time. Machine learning is used in a wide range of applications, such as image and speech recognition, natural language processing, recommendation systems, and a

'Sure! There are numerous machine learning algorithms, but I can mention a few popular ones:\n\n1. Linear Regression: This algorithm is used to model the relationship between a dependent variable and one or more independent variables by fitting a linear equation to observed data.\n\n2. Logistic Regression: It is a classification algorithm used when the dependent variable is categorical. It predicts the probability of an event occurring based on independent variables.\n\n3. Support Vector Machines (SVM): SVM is a binary classification algorithm that finds an optimal hyperplane in a high-dimensional feature space to separate different classes.\n\n4. Decision Trees: This algorithm uses a tree-like model to make decisions based on splitting the data into various branches, considering different features at each node. It is commonly used for both classification and regression tasks.\n\n5. Random Forest: Random Forest is an ensemble learning method that combines multiple decision trees to mak

### you can see it's stacking the conversation in the prompt

In [11]:
# find whole memory buffer

print(conversation.memory.buffer)

Human: Hi there! I am Zuu
AI: Hello, Zuu! I'm an AI designed to have conversations with people. How can I assist you today?
Human: Do you know machine learning?
AI: Yes, I am familiar with machine learning. Machine learning is a subset of artificial intelligence that focuses on the development of algorithms and models that allow computers to learn and make predictions or decisions without being explicitly programmed. It involves the use of large amounts of data to train models and improve their performance over time. Machine learning is used in a wide range of applications, such as image and speech recognition, natural language processing, recommendation systems, and autonomous vehicles, to name just a few.
Human: Name few algorithms ?
AI: Sure! There are numerous machine learning algorithms, but I can mention a few popular ones:

1. Linear Regression: This algorithm is used to model the relationship between a dependent variable and one or more independent variables by fitting a linear

## ConversationSummaryMemory

In [27]:
llm_sum = ChatOpenAI(
                    openai_api_key=os.environ["OPENAI_API_KEY"],
                    engine = os.environ["ENGINE"],
                    model='gpt-3.5-turbo',
                    temperature=0.9, 
                    max_tokens = 256
                    )
summary_memory = ConversationSummaryMemory(llm=llm_sum) # llm=llm_sum used to get the summary of the conversation
conversation = ConversationChain(
                                llm=llm, 
                                verbose=True, 
                                memory=summary_memory
                                )

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


In [28]:
conversation.predict(input="Hi there! I am Zuu")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there! I am Zuu
AI:

> Finished chain.


'Hello Zuu! Nice to meet you. How can I assist you today?'

In [29]:
conversation.predict(input="do you know machine learning ?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
The human introduces themselves as Zuu and the AI greets them. The AI asks Zuu how it can assist them.
Human: do you know machine learning ?
AI:

> Finished chain.


'Yes, I know about machine learning. Machine learning is a subfield of artificial intelligence that focuses on the development of algorithms and models that allow computers to automatically learn and improve from experience without being explicitly programmed. It involves teaching computers to learn patterns and make predictions or decisions based on data. Machine learning is used in various applications such as image recognition, natural language processing, and recommendation systems. Is there anything specific you would like to know about machine learning, Zuu?'

### the highlighted part represent the summary of the conversation
![Alt text](assets/image.png)

In [30]:
conversation.predict(input="Name few algorithms ?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
The human introduces themselves as Zuu and the AI greets them. The AI asks Zuu how it can assist them and Zuu asks if the AI knows about machine learning. The AI responds that it is familiar with machine learning, explaining that it is a subfield of artificial intelligence that focuses on algorithms and models that allow computers to learn and improve from experience. It is used in various applications such as image recognition and recommendation systems. The AI asks Zuu if there is anything specific they would like to know about machine learning.
Human: Name few algorithms ?
AI:

> Finished chain.


"There are numerous algorithms used in machine learning, each designed to solve different types of problems. Some popular examples include:\n\n1. Linear regression: This algorithm is used for predicting continuous output variables, given a set of input variables that have a linear relationship with the output.\n\n2. Logistic regression: Similar to linear regression, but used for predicting binary or categorical output variables.\n\n3. Decision trees: These algorithms create a tree-like model of decisions and their possible consequences. They are commonly used in classification tasks.\n\n4. Random forests: This algorithm uses an ensemble of decision trees to make predictions. It combines multiple decision trees to reduce overfitting and increase accuracy.\n\n5. Support Vector Machines (SVM): SVM is a powerful algorithm used for both classification and regression tasks. It separates different classes by finding the best hyperplane that maximizes the margin between them.\n\n6. Naive Bayes

## ConversationBufferWindowMemory

In [32]:
window_memory = ConversationBufferWindowMemory(k=2)
conversation = ConversationChain(
                                llm=llm, 
                                verbose=True, 
                                memory=window_memory
                                )

In [33]:
conversation.predict(input="Hi there! I am Zuu")    



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there! I am Zuu
AI:

> Finished chain.


'Hello Zuu! How can I assist you today?'

In [34]:
conversation.predict(input="Do you know machine learning?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there! I am Zuu
AI: Hello Zuu! How can I assist you today?
Human: Do you know machine learning?
AI:

> Finished chain.


'Yes, I am familiar with machine learning. Machine learning is a subfield of artificial intelligence that focuses on developing algorithms and statistical models that allow computers to learn and make predictions or decisions without being explicitly programmed. It is a highly dynamic and rapidly evolving field with various techniques and approaches, such as supervised learning, unsupervised learning, and reinforcement learning. Machine learning has applications in many domains, including natural language processing, computer vision, and data analysis. Is there anything specific you would like to know about machine learning?'

In [35]:
conversation.predict(input="Name few algorithms ?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there! I am Zuu
AI: Hello Zuu! How can I assist you today?
Human: Do you know machine learning?
AI: Yes, I am familiar with machine learning. Machine learning is a subfield of artificial intelligence that focuses on developing algorithms and statistical models that allow computers to learn and make predictions or decisions without being explicitly programmed. It is a highly dynamic and rapidly evolving field with various techniques and approaches, such as supervised learning, unsupervised learning, and reinforcement learning. Machine learning has applications in many domains, including natural language processing, computer vision, and data analysis. Is there 

"Certainly! There are many machine learning algorithms, but here are a few popular ones:\n\n1. Linear Regression: This algorithm is used for predicting a continuous output variable based on one or more input variables. It fits a linear equation to the data and finds the best fit line.\n\n2. Logistic Regression: Unlike linear regression, this algorithm is used for classification problems. It predicts the probability of an instance belonging to a particular class by fitting a logistic function to the data.\n\n3. Support Vector Machines (SVM): SVM is a powerful algorithm for both classification and regression tasks. It finds the optimal hyperplane that separates the data points of different classes with the largest margin.\n\n4. Decision Trees: These are tree-like models where each internal node represents a feature, each branch represents a decision rule, and each leaf node represents the outcome. They are widely used for classification and regression problems.\n\n5. Random Forest: This 

In [36]:
conversation.predict(input="Thanks")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Do you know machine learning?
AI: Yes, I am familiar with machine learning. Machine learning is a subfield of artificial intelligence that focuses on developing algorithms and statistical models that allow computers to learn and make predictions or decisions without being explicitly programmed. It is a highly dynamic and rapidly evolving field with various techniques and approaches, such as supervised learning, unsupervised learning, and reinforcement learning. Machine learning has applications in many domains, including natural language processing, computer vision, and data analysis. Is there anything specific you would like to know about machine learning?
Huma

"You're welcome! If you have any more questions or if there's anything else I can help you with, feel free to ask!"

### now you can see the conversation is buffered with window as it removed my first query

## ConversationSummaryBufferMemory

In [37]:
memory = ConversationSummaryBufferMemory(
                                        llm=llm_sum, 
                                        max_token_limit=40 # instead of setting window of interaction, we can set the max token limit
                                        ) 

conversation = ConversationChain(
                                llm=llm, 
                                verbose=True, 
                                memory=memory
                                )

In [38]:
conversation.predict(input="Hi there! I am Zuu")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there! I am Zuu
AI:

> Finished chain.


"Hello Zuu! It's a pleasure to meet you. How can I assist you today?"

In [39]:
conversation.predict(input="Do you know machine learning?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there! I am Zuu
AI: Hello Zuu! It's a pleasure to meet you. How can I assist you today?
Human: Do you know machine learning?
AI:

> Finished chain.


'Yes, I do! Machine learning is a branch of artificial intelligence that focuses on the development of algorithms and models that allow computers to learn and make predictions or decisions without being explicitly programmed. It involves training a computer with a large amount of data and allowing it to learn patterns and relationships within the data to make accurate predictions or decisions in the future. Machine learning is used in various fields such as image recognition, natural language processing, recommendation systems, and autonomous vehicles, among others.'

In [40]:
conversation.predict(input="Name few algorithms ?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human introduces themselves as Zuu and the AI greets them. Zuu asks if the AI knows about machine learning. The AI responds by explaining that machine learning is a branch of artificial intelligence that involves training computers with data to allow them to learn and make predictions or decisions. Machine learning is used in various fields such as image recognition, natural language processing, recommendation systems, and autonomous vehicles.
Human: Name few algorithms ?
AI:

> Finished chain.


"There are several popular machine learning algorithms that are commonly used in various applications. Here are a few examples:\n\n1. Linear Regression: This algorithm is used for predicting a continuous outcome by fitting a linear relationship between the input features and the target variable.\n\n2. Logistic Regression: It is a statistical algorithm used for binary classification problems. It predicts the probability of an instance belonging to a particular class.\n\n3. Decision Trees: This algorithm creates a flowchart-like structure to make decisions based on the input features. It can handle both categorical and numerical data.\n\n4. Random Forest: It is an ensemble learning algorithm that combines multiple decision trees to make predictions. It is known for its better accuracy and robustness.\n\n5. Support Vector Machines (SVM): SVM is a popular algorithm for both classification and regression tasks. It finds an optimal hyperplane to separate different classes in the input data.\

In [41]:
conversation.predict(input="Thanks")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human introduces themselves as Zuu and the AI greets them. Zuu asks if the AI knows about machine learning. The AI responds by explaining that machine learning is a branch of artificial intelligence that involves training computers with data to allow them to learn and make predictions or decisions. Machine learning is used in various fields such as image recognition, natural language processing, recommendation systems, and autonomous vehicles. Zuu then asks the AI to name a few machine learning algorithms. The AI provides examples such as linear regression, logistic regression, decision trees, random forest, support vector machines, k-means clustering, naiv

"You're welcome! Is there anything else you would like to know about machine learning or any other topic? I'm here to help!"

## Conversation Knowledge Graph Memory

In [44]:
template = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:

{history}

Conversation:
Human: {input}
AI:"""
prompt = PromptTemplate(
    input_variables=["history", "input"], template=template
)

conversation_with_kg = ConversationChain(
                                        llm=llm, 
                                        verbose=True, 
                                        prompt=prompt,
                                        memory=ConversationKGMemory(llm=llm)
                                    )

In [45]:
conversation_with_kg.predict(input="Hi there! I am Zuu")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:



Conversation:
Human: Hi there! I am Zuu
AI:

> Finished chain.


"Hello Zuu! It's nice to meet you. How can I assist you today?"

In [46]:
conversation_with_kg.predict(input="Do you know machine learning?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:



Conversation:
Human: Do you know machine learning?
AI:

> Finished chain.


'Yes, I am familiar with machine learning. It is a branch of artificial intelligence that focuses on the development of algorithms and models that allow computers to learn from and make predictions or decisions based on data. It involves techniques such as regression, classification, clustering, and deep learning. Machine learning has applications in various fields including image and speech recognition, natural language processing, and recommendation systems.'

In [47]:
conversation_with_kg.predict(input="Name few algorithms ?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:



Conversation:
Human: Name few algorithms ?
AI:

> Finished chain.


"There are numerous algorithms used in various fields. Some commonly known algorithms include:\n\n1. Bubble Sort: It is a simple sorting algorithm that repeatedly compares adjacent elements and swaps them if they are in the wrong order.\n\n2. Binary Search: It is an efficient searching algorithm that finds the position of a target value within a sorted array by repeatedly dividing the search interval in half.\n\n3. Dijkstra's Algorithm: It is a popular algorithm used for finding the shortest path between nodes in a graph with non-negative edge weights.\n\n4. Random Forest: It is an ensemble learning method that combines multiple decision trees to make predictions.\n\n5. K-means Clustering: It is an algorithm used for partitioning a set of data points into K clusters based on their similarity.\n\n6. PageRank: It is an algorithm used by Google to rank web pages in their search engine results.\n\n7. Support Vector Machines (SVM): It is a supervised learning algorithm used for classificati

In [48]:
print(conversation_with_kg.memory.kg)
print(conversation_with_kg.memory.kg.get_triples())

[('Zuu', 'Hi there', 'is'), ('Bubble Sort', 'sorting algorithm', 'is a'), ('Binary Search', 'searching algorithm', 'is a'), ("Dijkstra's Algorithm", 'algorithm', 'is a'), ('Random Forest', 'ensemble learning method', 'is an'), ('K-means Clustering', 'algorithm', 'is an'), ('PageRank', 'algorithm', 'is an'), ('Support Vector Machines (SVM)', 'learning algorithm', 'is a'), ('Neural Networks', 'set of algorithms', 'are a')]
